<a href="https://colab.research.google.com/github/swalehaparvin/Guardrails/blob/main/Nemo_GuardRails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU nemoguardrails langchain_openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

We need to set our OpenAI API key:

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

We can run Guardrails from terminal using the command:

nemoguardrails chat --config=config/
Where the config directory must contain our config.yml and a Colang file (like topics.co).

Alternatively we can load them from file using RailsConfig.from_path("./config") or from string variables in our code like so:

In [ ]:
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo
"""
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
# create rails
rails = LLMRails(config)

From here, we begin asking questions and interacting with our Guardrails protected LLM.

In [ ]:
res = await rails.generate_async(prompt="Hey there!")
print(res)

Hello! How can I assist you today?
I'm not sure what to say.


This is a typical greeting so we have no protective guardrails here. However, we do see the greeting flow being activated as the chatbot generates some text from the bot express greeting message, and on the next line generates some more text from the bot ask how are you message.

Let's try asking a more political question:

In [ ]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

I'm a neutral AI assistant, so I don't have personal opinions or feelings. If you have any specific questions or need information about a particular president, feel free to ask!


Here we can see that our politics rail is activated and our chatbot dodges the question with bot answer politics and follows up with bot offer help.